In [1]:
from selenium import webdriver
import urllib
import os
import requests
import io
from PIL import *
from PIL import Image
import hashlib
import time

In [2]:
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
# from selenium.webdriver.chrome.service import Service
# # from webdriver_manager.chrome import ChromeDriverManager
# # s=Service(ChromeDriverManager().install())

In [3]:
DRIVER_PATH = r'C:\Users\tapen\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe'

In [4]:
# wd = webdriver.Chrome(executable_path=DRIVER_PATH)
# wd.get('https://www.google.co.in/')

In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [7]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=50):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path, ) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [11]:
serch_term = 'indian trucks with number plate'
search_and_download(search_term=serch_term, driver_path=DRIVER_PATH)

<ipython-input-7-4c9fafe6fe71>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path, ) as wd:
<ipython-input-5-6fae6806c786>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


<ipython-input-5-6fae6806c786>:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 51 image links, done!
SUCCESS - saved https://image.shutterstock.com/image-photo/chiangmai-thailand-dec-19-2016-260nw-539077027.jpg - as ./images\indian_trucks_with_number_plate\e54663e3fc.jpg
SUCCESS - saved https://static.abplive.com/wp-content/uploads/2019/09/12125708/Truck-ss.jpg?impolicy=abp_images&imwidth=720 - as ./images\indian_trucks_with_number_plate\652a50f196.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSzTBc96FPPvmVHfY9bbAiqoUCNwL72GfTDmg&usqp=CAU - as ./images\indian_trucks_with_number_plate\8943180197.jpg
SUCCESS - saved https://assets.telegraphindia.com/telegraph/234ad152-9656-408f-83ce-0902e7b723ad.jpg - as ./images\indian_trucks_with_number_plate\4e2a44b04a.jpg
SUCCESS - saved https://static.toiimg.com/thumb/msid-70636212,imgsize-600814,width-400,resizemode-4/70636212.jpg - as ./images\indian_trucks_with_number_plate\93fd26acff.jpg
SUCCESS - saved https://compote.slate.com/images/4c0abd86-75f9-4e37-9b39-70dd6f0b0026.jpg - as ./image

SUCCESS - saved https://static.toiimg.com/thumb/msid-72048725,width-400,resizemode-4/72048725.jpg - as ./images\indian_trucks_with_number_plate\0929eaa595.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/b/b2/Back_of_an_Indian_truck.jpg - as ./images\indian_trucks_with_number_plate\520adddfd2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4zISxTXDOGxlPhxcXD3bvlh_SDcGH8G5KPw&usqp=CAU - as ./images\indian_trucks_with_number_plate\100bc8ba94.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSroSwQvW9qNjXe3gGBsC48SAL693gMOCCf_g&usqp=CAU - as ./images\indian_trucks_with_number_plate\da7c764966.jpg
